In [9]:
import logging
import requests
import pyodbc
import os
import io
# import azure.functions as func
from datetime import datetime, date, timedelta
import time
import json
import gzip
import pandas as pd
import traceback
import numpy as np

idag = datetime.now()
igår = idag - timedelta(days=1)
år = idag.year
måned = idag.month
dag = idag.day
if måned <= 7:
    termin = "VÅR"
else:
    termin = "HØST"

CD2_base_url = "https://api-gateway.instructure.com"
CD2_client_id = "eu-west-1#1505cc4e-f9aa-41d5-a364-214699ebe66b" # os.environ['CD2_client_id']
CD2_client_secret = "H0BAdlBVfSXps5KmKNFEu9e_fXhpDGc_ksjD0Fm2cH4" # os.environ['CD2_client_secret']
conn_str = os.environ["Connection_SQL"]

In [14]:
def akv_hent_CD2_access_token():
    be_om_access_token = requests.request(
        "POST",
        "https://api-gateway.instructure.com/ids/auth/login",
        data={'grant_type': 'client_credentials'},
        auth=(CD2_client_id, CD2_client_secret)
        )
    if be_om_access_token.status_code == 200:
        CD2_access_token = be_om_access_token.json()['access_token']
        return CD2_access_token
    else:
        feilmelding = f"Klarte ikkje å skaffe access_token, feil {be_om_access_token.status_code}"
        logging.error(feilmelding)
        return feilmelding


def akv_finn_sist_oppdatert(tabell):
    if os.path.exists(f"sist_oppdatert_{tabell}.txt"):
        with open(f"sist_oppdatert_{tabell}.txt", "r") as f_in:
            return f_in.read()
    else:
        return (datetime.now()-timedelta(days=1)).isoformat() + "Z"


def akv_lagre_sist_oppdatert(tabell, sist_oppdatert):
    with open(f"sist_oppdatert_{tabell}.txt", "w") as f:
        f.write(sist_oppdatert)

In [11]:
def akv_hent_CD2_fil(innfil, token, svar):
    try:
        requesturl = "https://api-gateway.instructure.com/dap/object/url"
        payload = f"{svar['objects']}"
        payload = payload.replace('\'', '\"')
        headers = {'x-instauth': token, 'Content-Type': 'text/plain'}
        respons = requests.request("POST", requesturl, headers=headers, data=payload)
        respons.raise_for_status()
        fil = respons.json()
        url = fil['urls'][innfil]['url']
        data = requests.request("GET", url)
        buffer = io.BytesIO(data.content)
        with gzip.GzipFile(fileobj=buffer, mode='rb') as utpakka_fil:
            utpakka_data = utpakka_fil.read().decode()
        return utpakka_data
    except requests.exceptions.RequestException as exc:
        raise exc

In [12]:
def akv_les_CD2_tabell(tabell):
    CD2_access_token = akv_hent_CD2_access_token()
    headers = {'x-instauth': CD2_access_token, 'Content-Type': 'text/plain'}
    payload = '{"format": "csv", "since": \"%s\"}' % (forrige_oppdatering)
    requesturl = f"https://api-gateway.instructure.com/dap/query/canvas/table/{tabell}/data"
    logging.info(f"Sender søk til {requesturl}")
    try:
        start_finn_filar = time.perf_counter()
        r = requests.request("POST", requesturl, headers=headers, data=payload)
        r.raise_for_status()
        respons = r.json()
        id = respons['id']
        vent = True
        while vent:
            logging.info(f"Vent til fila blir tilgjengelig: {requesturl}/job/{id}")
            requesturl2 = f"https://api-gateway.instructure.com/dap//job/{id}"
            r2 = requests.request("GET", requesturl2, headers=headers)
            time.sleep(5)
            respons2 = r2.json()
            if respons2['status'] == "complete":
                logging.info(f"Fila(ne) er tilgjengeleg(e): {requesturl}/job/{id}")
                vent = False
                filer = respons2['objects']
                sist_oppdatert = respons2['until']
        logging.info(f"Totalt for {tabell}_finn_filer: {time.perf_counter() - start_finn_filar}")
    except requests.exceptions.RequestException as exc:
        raise exc
    dr_liste = []
    start_les_filer = time.perf_counter()
    for fil in filer:
        data = io.StringIO(akv_hent_CD2_fil(fil['id'], CD2_access_token, respons2))
        df = pd.read_csv(data, sep=",")
        dr_liste.append(df)
    alledata = pd.concat(df for df in dr_liste if not df.empty)
    logging.info(f"Totalt for {tabell}_les_filer: {time.perf_counter() - start_les_filer}")
    return alledata, sist_oppdatert

In [16]:
tabell = "enrollments"
logging.basicConfig(filename=f'{tabell}.log', encoding='utf-8', level=logging.INFO)
forrige_oppdatering = akv_finn_sist_oppdatert(tabell)
alledata, sist_oppdatert = akv_les_CD2_tabell(tabell)
akv_lagre_sist_oppdatert(tabell, sist_oppdatert)
alledata.to_csv("alledata_enrollment.csv", index=False)
enrollments = alledata[['key.id', 'value.user_id', 'value.course_id', 'value.type', 'value.created_at', 'value.updated_at', 'value.start_at', 'value.end_at', 'value.workflow_state', 'value.total_activity_time', 'value.last_activity_at']]
nye = enrollments[enrollments['value.created_at'] > forrige_oppdatering]

# For testing lokalt. Kan fjernes etter testing.
nye.to_csv(f"nye_{tabell}_{sist_oppdatert[0:10]}.csv", index=False)

/tmp/ipykernel_495/1008012460.py:32: DtypeWarning: Columns (14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data, sep=",")


In [22]:
with pyodbc.connect(conn_str) as cnxn:
    with cnxn.cursor() as cursor:
        for index, row in nye.iterrows():
            course_id= row['value.course_id'],
            enrollment_id= row['key.id'],
            user_id= row['value.user_id'],
            sis_user_id= "",
            enrollment_type= row['value.type'],
            created_at= row['value.created_at'],
            updated_at= row['value.updated_at'],
            enrollment_state= str(row['value.workflow_state']),
            total_activity_time= str(row['value.total_activity_time']),
            last_activity_at= str(row['value.last_activity_at'])
            try:
                merge_query = """
                                MERGE INTO [stg].[Canvas_Enrollments] AS target
                                USING (SELECT ?, ?, ?, ?, ?, CONVERT(datetime, ?, 127), CONVERT(datetime, ?, 127), ?, ?, CONVERT(datetime, ?, 127)) AS source (
                                    [enrollment_id],
                                    [user_id],
                                    [sis_user_id],
                                    [course_id],
                                    [type],
                                    [created_at],
                                    [updated_at],
                                    [enrollment_state],
                                    [total_activity_time],
                                    [last_activity_at]
                                )
                                ON target.[enrollment_id] = source.[enrollment_id]
                                WHEN MATCHED THEN
                                    UPDATE SET target.[user_id]= source.[user_id],
                                        target.[sis_user_id] = source.[sis_user_id],
                                        target.[course_id] = source.[course_id],
                                        target.[type] = source.[type],
                                        target.[created_at] = source.[created_at],
                                        target.[updated_at] = source.[updated_at],
                                        target.[enrollment_state] = source.[enrollment_state],
                                        target.[total_activity_time] = source.[total_activity_time],
                                        target.[last_activity_at] = source.[last_activity_at]
                                WHEN NOT MATCHED THEN
                                    INSERT ([enrollment_id],
                                        [user_id],
                                        [sis_user_id],
                                        [course_id],
                                        [type],
                                        [created_at],
                                        [updated_at],
                                        [enrollment_state],
                                        [total_activity_time],
                                        [last_activity_at]
                                    )
                                    VALUES (source.[enrollment_id],
                                        source.[user_id],
                                        source.[sis_user_id],
                                        source.[course_id],
                                        source.[type],
                                        source.[created_at],
                                        source.[updated_at],
                                        source.[enrollment_state],
                                        source.[total_activity_time],
                                        source.[last_activity_at]
                                    );
                            """
                cursor.execute(merge_query, (enrollment_id, user_id, sis_user_id, course_id, enrollment_type, created_at, updated_at, enrollment_state, total_activity_time, last_activity_at))
            except Exception as e:
                print(f"Error inserting row {index}: {e}")
        cnxn.commit()

Error inserting row 187586: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187587: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187588: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187589: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187590: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187591: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187592: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187593: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187594: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187595: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187596: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187597: ("A TVP's rows must be Sequence objects.", 'HY000')
Error inserting row 187598: ("A TVP's ro

In [19]:
entity

{'course_id': '29020',
 'enrollment_id': '1453477',
 'user_id': '109225',
 'sis_user_id': '',
 'type': 'StudentEnrollment',
 'created_at': '2024-10-30T04:57:53.611Z',
 'updated_at': '2024-10-30T04:57:53.611Z',
 'enrollment_state': 'active',
 'total_activity_time': 'nan',
 'last_activity_at': 'nan'}